In [3]:
# libaries 

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.llms import CTransformers
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama
from random import getrandbits
from transformers import AutoTokenizer
from IPython.display import display, HTML
import json
import time
import pathlib
import os

In [4]:
# Reading txt files

# define what documents to load and where from
class UTF8TextLoader(TextLoader):
    def __init__(self, file_path: str, encoding: str = 'utf-8'):
        super().__init__(file_path, encoding)
        
loader = DirectoryLoader("./", glob="*.txt", loader_cls=UTF8TextLoader)

# interpret information in the documents
documents = loader.load()
splitter = CharacterTextSplitter()
texts = splitter.split_documents(documents)
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'})

# create and save the local database
db = FAISS.from_documents(texts, embeddings)
db.save_local("faiss")

z:\Uni-CE901\.venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
z:\Uni-CE901\.venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
# Set the path to where you have the text files
directory_path = r"Z:\Uni-CE901\22-24_CE901-CE911-CF981-SU_stoyles_thomas\Data" # Change this to your directory

# Initialize the tokenizer 
tokenizer = AutoTokenizer.from_pretrained("gpt2") # Change if you wish to test other tokenizers

def count_tokens_in_file(file_path, tokenizer):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    tokens = tokenizer.encode(text)
    return len(tokens)

def check_tokens_in_directory(directory_path, tokenizer, max_tokens=512):
    for file_name in os.listdir(directory_path):
        if file_name.endswith('.txt'):
            file_path = os.path.join(directory_path, file_name)
            num_tokens = count_tokens_in_file(file_path, tokenizer)
            print(f"File: {file_name}, Tokens: {num_tokens}")
            if num_tokens > max_tokens:
                print(f"WARNING: {file_name} exceeds the maximum token limit of {max_tokens}")

# Check token counts in the directory
check_tokens_in_directory(directory_path, tokenizer, max_tokens=512)


Token indices sequence length is longer than the specified maximum sequence length for this model (1231 > 1024). Running this sequence through the model will result in indexing errors


File: Anxiety_Causes.txt, Tokens: 271
File: Anxiety_HelpingFriends.txt, Tokens: 950
File: Anxiety_PanicAttacks.txt, Tokens: 117
File: Anxiety_symptoms.txt, Tokens: 204
File: Anxiety_WhatToDo.txt, Tokens: 262
File: Anxiety_WhatToSay.txt, Tokens: 399
File: Depression.txt, Tokens: 1231
File: Depression_Causes.txt, Tokens: 774
File: Depression_HelpingFriends.txt, Tokens: 767
File: Depression_HowToTell&Living.txt, Tokens: 213
File: Depression_Symptoms.txt, Tokens: 271
File: Depression_Treatment&Doctor.txt, Tokens: 207
File: Depression_Treatments.txt, Tokens: 1231
File: Depression_Types.txt, Tokens: 519
File: Depression_WhatToSay.txt, Tokens: 512
File: Depression_WhenToSeeDoctor&Treatments.txt, Tokens: 207
File: Depression_WhenToSeeDoctor.txt, Tokens: 207
File: Stress_Causes.txt, Tokens: 373
File: Stress_HelpingFriends.txt, Tokens: 491
File: Stress_Symptoms.txt, Tokens: 427
File: Stress_WhatIsIt.txt, Tokens: 128
File: Stress_WhatToDo.txt, Tokens: 770


In [6]:
def truncate_tokens(text, max_tokens, tokenizer):
    tokens = tokenizer.encode(text, truncation=True, max_length=max_tokens)
    return tokenizer.decode(tokens[:max_tokens])

In [7]:
# prepare the template we will use when prompting the response
template = """Context: {context}
Question: {question}
"""
# load the language model (Editing model to find best and tokens)
config = {'max_new_tokens': 512, 'temperature': 0.01}
llm = Ollama(model="llama3") # Use Ollama website for more models https://ollama.com/library

# Enable dangerous deserialization if available
if hasattr(llm, 'allow_dangerous_deserialization'):
    llm.allow_dangerous_deserialization = True

# load the interpreted information from the local database
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'})
db = FAISS.load_local("faiss", embeddings, allow_dangerous_deserialization=True)

# prepare a version of the llm pre-loaded with the local content
retriever = db.as_retriever(search_kwargs={'k': 5}) # Max number of documents to read (k)
prompt = PromptTemplate(
    template=template,
    input_variables=['context', 'question'])

QA_LLM = RetrievalQA.from_chain_type(llm=llm,
                                     chain_type='stuff',
                                     retriever=retriever,
                                     return_source_documents=True,
                                     chain_type_kwargs={'prompt': prompt})

z:\Uni-CE901\.venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
# Defining query

def query(models, question):
    model_names = {}
    model_results = {}

    for model in models:
        model_path = model.combine_documents_chain.llm_chain.llm.model
        model_name = pathlib.Path(model_path).name
        
        # Generate a random key for storing results
        rand_key = getrandbits(32)
        
        time_start = time.time()
        output = model({'query': question})
        response = output["result"]
        time_elapsed = time.time() - time_start

        # Store the model name and result in dictionaries
        model_names[rand_key] = model_name
        model_results[rand_key] = response

        # still display each model's response time if needed
        display(HTML(f'<code>{model_name} response time: {time_elapsed:.02f} sec</code>'))

    # Randomize the order of model results to avoid bias
    model_results = dict(sorted(model_results.items()))

    return model_names, model_results

In [9]:
# Picking best model from the chosen options

def evaluate_responses(model_results):
    print("Please evaluate the following responses and pick the best one:")
    
    for key, response in model_results.items():
        print(f"Response {key}: {response}")
        print("-" * 80)
    
    chosen_key = int(input("Enter the key of the best response: "))
    return chosen_key

In [10]:
# What to say when model choosen
def main(models, question):
    model_names, model_results = query(models, question)
    chosen_key = evaluate_responses(model_results)
    
    print(f"The chosen model is: {model_names[chosen_key]}")

In [11]:
# Instantiate the models separately
QA_LLM1 = RetrievalQA.from_chain_type(
    llm=Ollama(model="llama3"),
    chain_type='stuff',
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={'prompt': prompt}
)

QA_LLM2 = RetrievalQA.from_chain_type(
    llm=Ollama(model="llama3.1"),
    chain_type='stuff',
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={'prompt': prompt}
)

# List of models to evaluate
models = [QA_LLM1, QA_LLM2]

# The question you wish to ask
question = "What are the symptoms of Depression?"

# Run the evaluation
main(models, question)


z:\Uni-CE901\.venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Please evaluate the following responses and pick the best one:
Response 3141402879: According to the text, the symptoms of Depression can be categorized into three types:

1. **Psychological Symptoms**:
	* Continuous low mood or sadness
	* Feeling hopeless and helpless
	* Having low self-esteem
	* Feeling tearful
	* Feeling guilt-ridden
	* Feeling irritable and intolerant of others
	* Having no motivation or interest in things
	* Finding it difficult to make decisions
	* Not getting any enjoyment out of life
	* Feeling anxious or worried
	* Having suicidal thoughts or thoughts of harming yourself
2. **Physical Symptoms**:
	* Moving or speaking more slowly than usual
	* Changes in appetite or weight (usually decreased, but sometimes increased)
	* Constipation
	* Unexplained aches and pains
	* Lack of energy
	* Low sex drive (loss of libido)
	* Disturbed sleep – for example, finding it difficult to fall asleep at night or waking up very early in the morning
3. **Psychological Symptoms th

KeyError: 1